<a href="https://colab.research.google.com/github/Shujaat123/Ear_Biometrics/blob/main/IITD_Dataset_CustomModel(Paper)_EarBiometrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Load useful packages
!pip install wget
!pip install py7zr

import keras
import py7zr
from zipfile import ZipFile
from random import sample

import PIL.Image as Image
from scipy import io
import matplotlib.pyplot as plt
from  sklearn.model_selection import train_test_split
from os import listdir
from os import path
import h5py
import keras.backend as K
import numpy as np
import tensorflow as tf
import wget
from keras import regularizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras.models import Model
from keras.models import load_model
from keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import confusion_matrix

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9675 sha256=77e2cc07d99dc6db40aeb98fb2b3361f38bd7c68b1e15fffc153dc0faaa19b6e
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
     |████████████████████████████████| 71kB 8.4MB/s 
     |████████████████████████████████| 358kB 34.1MB/s 
     |████████████████████████████████| 1.9MB 47.6MB/s 
     |████████████████████████████████| 122kB 51.0MB/s 
     |████████████████████████████████| 2.2MB 49.6MB/s 


In [24]:
data_path = 'https://github.com/Alishba18001778/EarClassification/blob/main/IITD_Data_Tensor.mat?raw=true'
Data_filename = 'IITD_Data_Tensor.mat'

data_path_labels = 'https://github.com/Alishba18001778/EarClassification/blob/main/IITD_Labels.mat?raw=true'
labels_filename = 'IITD_Labels.mat'

if(path.exists(labels_filename)):
  !rm $labels_filename
  print('existing file:', labels_filename, ' has been deleted')
print('downloading latest version of file:', labels_filename)
wget.download(data_path_labels, labels_filename)

if(path.exists(Data_filename)):
  !rm $Data_filename
  print('existing file:', Data_filename, ' has been deleted')
print('downloading latest version of file:', Data_filename)
wget.download(data_path, Data_filename)
# 
u = io.loadmat('IITD_Data_Tensor.mat')
v = io.loadmat('IITD_Labels.mat')
sub_labels = v["sub_labels"]
sub_labels = np.squeeze(sub_labels.transpose())
ear_images = u["ear_images"]
ear_images = np.array(ear_images)

# sub_labels = to_categorical(np.array(sub_labels))
print(ear_images.shape)
print(sub_labels.shape)

existing file: IITD_Labels.mat  has been deleted
downloading latest version of file: IITD_Labels.mat
existing file: IITD_Data_Tensor.mat  has been deleted
downloading latest version of file: IITD_Data_Tensor.mat
(793, 180, 50)
(793,)


In [30]:
X_train, X_test, y_train, y_test = train_test_split(ear_images, sub_labels, test_size=0.382093316519, stratify=sub_labels, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(490, 180, 50)
(490,)
(303, 180, 50)
(303,)


In [10]:
### Write Augmentation code here



In [62]:
#Custom Model

def new_model():
  input_img = Input(shape=(180,50, 1))
  num_filter = 8
  kernel_size = 3
  strides = 1
  
  x = Conv2D(num_filter, kernel_size, strides, activation='relu',padding = 'same', name = 'enc1')(input_img)
  x = MaxPool2D((2, 2))(x)

    
  x = Conv2D(2*num_filter, kernel_size, strides, activation='relu', padding = 'same', name = 'enc3')(x) 
  x = BatchNormalization()(x)
  x = MaxPool2D((2, 2))(x)   
 
    
  x = Conv2D(4*num_filter, kernel_size, strides, activation='relu', padding = 'same', name = 'enc4')(x) 
  x = MaxPool2D((2, 2))(x) 

  x = Conv2D(8*num_filter, kernel_size, strides, activation='relu', padding = 'same', name = 'enc5')(x) 
  x = BatchNormalization()(x)
  x = MaxPool2D((2, 2))(x) 
  
  x = Conv2D(16*num_filter, kernel_size, strides, activation='relu', padding = 'same', name = 'enc6')(x) 
  x = MaxPool2D((2, 2))(x) 

  
  x = Conv2D(32*num_filter, kernel_size, strides, activation='relu', padding = 'same', name = 'enc7')(x) 
  x = BatchNormalization()(x)

  y = Flatten()(x) 

  output = Dense(221, activation='softmax')(y)

  Arch_pre = Model(input_img, output)
  
  Arch_pre.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                loss=keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])
  return Arch_pre

Arch_pre= new_model()
Arch_pre.summary()

Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        [(None, 180, 50, 1)]      0         
_________________________________________________________________
enc1 (Conv2D)                (None, 180, 50, 8)        80        
_________________________________________________________________
max_pooling2d_95 (MaxPooling (None, 90, 25, 8)         0         
_________________________________________________________________
enc3 (Conv2D)                (None, 90, 25, 16)        1168      
_________________________________________________________________
batch_normalization_61 (Batc (None, 90, 25, 16)        64        
_________________________________________________________________
max_pooling2d_96 (MaxPooling (None, 45, 12, 16)        0         
_________________________________________________________________
enc4 (Conv2D)                (None, 45, 12, 32)        464

In [64]:
Arch_pre= new_model()
#Using the best model 
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, mode='min')

checkpoint = ModelCheckpoint('models\\modelweather-best.h5',
                                  verbose=0, monitor='val_loss',save_best_only=True, mode='auto')

Arch_pre.fit(X_train, y_train,
                epochs=100,
                batch_size=5,
                shuffle=True,
                validation_data=(X_test, y_test),
                callbacks = [es, checkpoint]
                )

del Arch_pre  # deletes the existing model
Arch_pre = load_model('models\\modelweather-best.h5')

Epoch 1/100
98/98 [==============================] - 3s 13ms/step - loss: 6.4293 - accuracy: 0.0112 - val_loss: 5.6007 - val_accuracy: 0.0132
Epoch 2/100
98/98 [==============================] - 1s 8ms/step - loss: 5.0741 - accuracy: 0.0628 - val_loss: 6.6437 - val_accuracy: 0.0066
Epoch 3/100
98/98 [==============================] - 1s 8ms/step - loss: 3.3353 - accuracy: 0.2639 - val_loss: 6.3175 - val_accuracy: 0.0066
Epoch 4/100
98/98 [==============================] - 1s 6ms/step - loss: 1.5159 - accuracy: 0.6590 - val_loss: 5.5703 - val_accuracy: 0.0066
Epoch 5/100
98/98 [==============================] - 1s 6ms/step - loss: 0.4649 - accuracy: 0.9116 - val_loss: 2.0347 - val_accuracy: 0.5413
Epoch 6/100
98/98 [==============================] - 1s 6ms/step - loss: 0.1573 - accuracy: 0.9730 - val_loss: 1.6278 - val_accuracy: 0.6007
Epoch 7/100
98/98 [==============================] - 1s 6ms/step - loss: 0.0533 - accuracy: 0.9891 - val_loss: 1.5154 - val_accuracy: 0.6733
Epoch 8/100


In [66]:
y_test_pred = Arch_pre.predict(X_test, batch_size=1, verbose=0)
acc = accuracy_score(y_test, y_test_pred.argmax(axis=1))

print('Test Accuracy: \t',acc)

Test Accuracy: 	 0.9141914191419142
